In [292]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [293]:
import pandas as pd
import numpy as np
from utilities import *
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline

In [294]:
from sklearn import set_config
set_config(transform_output="pandas")
set_config(display='diagram')

In [295]:
X_train, y_train, X_test, y_test = load_data()
selected_columns = np.load("./data/selected_columns.npy", allow_pickle=True)

In [296]:
from sklearn.ensemble import RandomForestRegressor

regresion_function = RandomForestRegressor()
column_dropper_transformer = ColumnDropperTransformer(selected_columns)
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', drop="if_binary", max_categories=5, sparse_output=False))
])

numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy="mean")),
    ('normalizer', MinMaxScaler())
])

column_transformer = ColumnTransformer(transformers=[
    ('cat', categorical_pipeline, make_column_selector(dtype_include=object)),
    ('num', numerical_pipeline, make_column_selector(dtype_exclude=object)),
])

model = Pipeline(steps=[
    ("column_dropper", column_dropper_transformer),
    ('column_transformer', column_transformer),
    ('regressor', regresion_function)
])

In [297]:
model.fit(X_train, y_train)

Pipeline(steps=[('column_dropper',
                 <utilities.ColumnDropperTransformer object at 0x78d447c4f6d0>),
                ('column_transformer',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore',
                                                                                 max_categories=5,
                                                                                 sparse_output=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x78d447c4d5d0>),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('normalizer',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x78d447c4d390>)])),
                ('regressor', RandomForestRegressor())])

In [298]:
prediction = model.predict(X_test)

/home/marcin/miniconda3/envs/DataMining/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [6, 7, 18, 22] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [299]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, prediction)

52926.77845677332